In [28]:
import numpy as np
import pandas as pd
import math
from tensorflow import keras
import tensorflow as tf

In [29]:
# Total RIS elements M:
M = 16
sigma = np.power(10,-8.5)       #-55
num_users = 4
# Coordinates of AP(x_ap, y_ap, z_ap):
AP = [0, 0, 0]

# Coordinates of RIS(x_r, y_r, z_r):
RIS = [300, 0, 0]

# Coordinates of user k(x_k, y_k, z_k):
users = []
for user in range(num_users):
    users.append([np.random.randint(-500, 800), np.random.randint(-500,500), np.random.randint(-500,500)])

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)

distance(AP, RIS), distance(AP, users[0]), distance(RIS, users[0])

(300.0, 463.09934139447876, 708.8448349251055)

In [30]:
# Calculating NLos vectors for v, gk:
v_nlos = np.random.normal(0, math.sqrt(0.5), size=(M,1)) + np.random.normal(0, math.sqrt(0.5), size=(M,1))
gk_nlos = np.random.normal(0, math.sqrt(0.5), size=(M,1)) + np.random.normal(0, math.sqrt(0.5), size=(M,1))
rk_nlos = np.random.normal(0, math.sqrt(0.5), size=(1,1)) + np.random.normal(0, math.sqrt(0.5), size=(1,1))


v_nlos = v_nlos.ravel().tolist()
gk_nlos = gk_nlos.ravel().tolist()
rk_nlos = rk_nlos.ravel().tolist()

In [31]:
# Generating v_los:
v_los = []

def sin_a_AR(x1,x2,y1,y2):
    return (y1-y2)/math.sqrt((x1-x2)**2 + (y1-y2)**2)

for m in range(1, M+1):
    v_los.append( math.cos((m-1)*math.pi*sin_a_AR(RIS[0], AP[0], RIS[1], AP[1])) 
              + (1j)*math.sin((m-1)*math.pi*sin_a_AR(RIS[0], AP[0], RIS[1], AP[1])))

In [32]:
# Generating gk_los:
gk_los = [[] for _ in range(num_users)]

for user in range(num_users):
    for m in range(1, M+1):
        gk_los[user].append( math.cos((m-1)*math.pi*sin_a_AR(users[user][0], RIS[0], users[user][1], RIS[1])) 
                         + (1j)*math.sin((m-1)*math.pi*sin_a_AR(users[user][0], RIS[0], users[user][1], RIS[1])))

In [33]:
# Generating v[n]:
v = []
rho_0 = 30       # actual rho_0 = -30 (using 1j later)
d_0 = 1
alpha = 3*(10**(-4))
K_AR = 3
K_RU = 3

def PL(d):
    # print(rho_0*((d/d_0)**alpha))
    return rho_0*((d/d_0)**alpha)

def generate_v():
    for m in range(1, M+1):
        v.append( 1j*math.sqrt(PL(300)) * (math.sqrt(K_AR/(K_AR+1)) *v_los[m-1] + math.sqrt(1/(K_AR+1)) *v_nlos[m-1]) )
    return v

In [34]:
# Generating g_k[n]:
gk = [[] for _ in range(num_users)]

def generate_gk():
    for user in range(num_users):
        for m in range(1, M+1):
            gk[user].append( 1j*math.sqrt(PL(distance(RIS, users[user]))) * (math.sqrt(K_RU/(K_RU+1)) *gk_los[user][m-1] + math.sqrt(1/(K_RU+1)) *gk_nlos[m-1]) )
    return gk   

In [35]:
rk = [[] for _ in range(num_users)]

def generate_rk():
    for user in range(num_users):
        rk[user].append( 1j*math.sqrt(PL(distance( AP, users[user]))) *rk_nlos[0])
    return rk

In [36]:
# Generating theta
def generate_theta():
    theta = []
    for m in range(M):
        theta.append(np.random.uniform(0, 2*math.pi, M))
    # theta.ravel().tolist()
    return theta
# theta_matrix = generate_theta()
# theta = [theta_matrix[i][i] for i in range(len(theta_matrix))]

In [37]:
gk = generate_gk()
v = generate_v()
rk = generate_rk()
# print(gk)
# print(v)
# print(rk)

user_power = np.power(10,-2.35)    # 24
# print(theta_0)
# print(len(rk[0]))
# print(len(theta_0))
# print(len(gk))
# df_u1 = pd.DataFrame(list(zip(gk, v, rk, theta_0)), columns=['gk', 'v', 'rk', 'theta_0'])

X = pd.DataFrame()
for user in range(num_users):
    dic = {}
    for i in range(len(gk[user])):
        dic['gk'+str(i)] = gk[user][i]
    for i in range(len(v)):
        dic['v'+str(i)] = v[i]
    dic['rk'] = rk[user]
    x = pd.DataFrame.from_dict(dic)

    X = pd.concat([X,x])

y = pd.DataFrame()
for user in range(num_users):
    theta_matrix = generate_theta()
    theta = [theta_matrix[i][i] for i in range(len(theta_matrix))]
    y_dic = {}
    for i in range(len(theta)):
        y_dic['theta'+str(i)] = [theta[i]]
    y_dic['pk'] = [user_power]
    # print(y_dic)
    y_df = pd.DataFrame.from_dict(y_dic)

    y = pd.concat([y, y_df])

print(X.shape, y.shape)
print(X.head())

(4, 33) (4, 17)
                  gk0                 gk1                 gk2  \
0  0.000000+5.974365j  4.568476+4.850805j  2.489313-3.538982j   
0  0.000000+5.973919j -4.546888+4.923256j -2.617015-3.457135j   
0  0.000000+5.974187j  1.077975+8.181064j  2.099648+4.762688j   
0  0.000000+5.973750j -4.589656+4.771247j -2.347997-3.622141j   

                  gk3                 gk4                  gk5  \
0 -3.212076-3.769431j -4.239538+1.438095j  0.901998+10.455188j   
0  3.040634-3.919014j  4.367088+1.162904j -0.527105+10.511514j   
0  3.011658+3.397824j  3.766372+2.191129j   4.324371+7.753701j   
0  3.388457-3.598071j  4.081479+1.725381j -1.300435+10.358991j   

                  gk6                 gk7                 gk8  \
0  4.731027-2.724159j  1.675887-3.728543j -3.817855+0.182296j   
0 -4.670469-2.273029j -2.161038-3.513501j  3.426658-0.281287j   
0  4.656511-2.198910j  4.745445+0.559810j  4.586528+1.777528j   
0 -4.746762-3.215212j -1.127935-3.897789j  4.169728+0.734771j   

 

In [38]:
class NeuralNetwork:
    def __init__(self):
        self.model = keras.models.Sequential()
        self.gk = X[[col for col in X if col.startswith('gk')]].to_numpy()
        self.v = X[[col for col in X if col.startswith('v')]].to_numpy()
        self.rk = X[[col for col in X if col.startswith('rk')]].to_numpy()

    def loss_func(self, y_true, y_pred):
        hk_final = 0
        for user in range(num_users):
            theta_1 = tf.linalg.diag(y_pred[:,:M][user])
            # pk = y_pred[:,-1]
            # pk_ = pk[0]
            # print(pk_)
            v_ = tf.transpose(tf.convert_to_tensor(self.v[user], dtype=tf.complex128))
            # print(v_)
            a = tf.math.conj(tf.convert_to_tensor(self.gk[user]))
            # print(a)
            theta_1= tf.cast(theta_1, tf.complex128)
            b = tf.tensordot(theta_1, v_, axes=1)
            b_ = tf.convert_to_tensor(b)
            # print(b_)
            rk_ = tf.convert_to_tensor(self.rk[user], dtype=tf.complex128)
            # print(rk_)
            c = tf.tensordot(a, b_, axes=1)
            c_ = tf.convert_to_tensor(c)
            # print(c_)
            hk = (tf.linalg.norm(rk_ + c_))**2
            # print(hk)
            hk_ = tf.fill((1,), hk)
            # print(hk_)
            hk_final = hk_final + hk_
            # print(hk_)
        return hk_final
        # RkO = 1/4*(np.log2( 1+ ((pk[0] * hk) * 4)/sigma**2 )) #4 -> users
        # rate = (pk*hk)
    # def loss_func(self, y_true, y_pred):
    #     tf.print(y_pred)
    #     squared_difference = tf.square(y_true - y_pred)
    #     return tf.reduce_mean(squared_difference, axis=-1)

    def InputLayer(self, Input):
        self.model.add(keras.layers.Flatten(input_shape=Input))
    def AddLayers(self, n_layers, neurons, activation):
        for i in range(n_layers):
            self.model.add(keras.layers.BatchNormalization())
            self.model.add(keras.layers.Dense(neurons[i], activation=activation[i],kernel_initializer='he_normal'))
            # self.model.add(keras.layers.Dropout(0.2))
            # print(self.model.summary())
    def fit(self,loss, X_train, y_train):
        # self.loss_func(X_train, y_train)
        optimizer = keras.optimizers.Adam(learning_rate=0.0003)
        self.model.compile(loss=self.loss_func, optimizer=optimizer, run_eagerly=True)
        self.model.fit(X_train, y_train, epochs=32)
    
    def predict(self, X):
        y = self.model.predict(X)
        return y

In [39]:
nmodel = NeuralNetwork()
nmodel.InputLayer([X.shape[1]])
nmodel.AddLayers(2,[30,y.shape[1]], ["relu","tanh"])
nmodel.fit("mse", X,y)
ypred = nmodel.predict(X)
print("power allocation = ",ypred[0][M])
print("theta[n] = ", ypred[0][:M])

Epoch 1/32
1/1 [==============================] - 0s 37ms/step - loss: 14735.5889
Epoch 2/32
1/1 [==============================] - 0s 28ms/step - loss: 15847.2480
Epoch 3/32
1/1 [==============================] - 0s 35ms/step - loss: 14591.2549
Epoch 4/32
1/1 [==============================] - 0s 31ms/step - loss: 12035.9619
Epoch 5/32
1/1 [==============================] - 0s 37ms/step - loss: 23528.5293
Epoch 6/32
1/1 [==============================] - 0s 34ms/step - loss: 10952.7695
Epoch 7/32
1/1 [==============================] - 0s 36ms/step - loss: 11387.7627
Epoch 8/32
1/1 [==============================] - 0s 40ms/step - loss: 10506.7871
Epoch 9/32
1/1 [==============================] - 0s 34ms/step - loss: 9972.6523
Epoch 10/32
1/1 [==============================] - 0s 28ms/step - loss: 9128.5781
Epoch 11/32
1/1 [==============================] - 0s 33ms/step - loss: 9377.5791
Epoch 12/32
1/1 [==============================] - 0s 29ms/step - loss: 8751.5781
Epoch 13/32
1/1 [

In [40]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, BatchNormalization, LSTM

# model = Sequential([
#     Dense(64, input_shape=(4,), activation="relu"),
#     LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1),
#     Dense(128, activation='relu'),
#     Dense(128, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(64, activation='relu'),
#     BatchNormalization(),
#     Dense(3, activation='tanh')
# ]);

# e = 32
# N = 16

# for epoch in range(e):
#     # Reset the environment and initial state
#     gk = generate_gk()
#     v = generate_v()
#     rk = generate_rk()
#     theta = generate_theta()

#     for n in range(N):
#     # Input the current phase shifts Θ[n − 1] and the CSI {gk[n], v[n], rk[n]} to the DNN
